# Hyperparameter Tuning
*(Note: This notebook runs significantly faster if you have access to a GPU. Use either the GPUHub, Google Colab, or your own GPU.)*

In this project, you will optimize the hyperparameters of a model in 3 stages.

## Paraphrase Detection
We finetune [distilbert-base-uncased](https://huggingface.co/distilbert-base-uncased) on [MRPC](https://huggingface.co/datasets/glue/viewer/mrpc/train), a paraphrase detection dataset. This notebook is adapted from a [PyTorch Lightning example](https://lightning.ai/docs/pytorch/1.9.5/notebooks/lightning_examples/text-transformers.html).

In [1]:
%pip install -q torch transformers lightning datasets wandb evaluate ipywidgets

Note: you may need to restart the kernel to use updated packages.


The next 4 cells are:
* Imports
* The `GLUEDataModule` loads the task's dataset and creates dataloaders for the train and valid sets.
* The `GLUETransformer` implements the model forward pass and the training/validation steps. You can check here what is logged with the `self.log` calls.
* The last cell runs training with the given parameters.

*Install the CLI and Python library for interacting with the Weights and Biases API.*

In [2]:
!wandb login

wandb: Currently logged in as: hslu-pascal-gansner (pascal-gansner-hochschule-luzern). Use `wandb login --relogin` to force relogin


In [3]:
from datetime import datetime
from typing import Optional

import wandb
import datasets
import evaluate
import lightning as L
import torch
from torch.optim import AdamW, Adam, SGD
from torch.utils.data import DataLoader
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
    get_constant_schedule_with_warmup
)
from pytorch_lightning.loggers import WandbLogger
from lightning.pytorch.callbacks import LearningRateFinder
import os

# Set the WANDB_NOTEBOOK_NAME environment variable
os.environ['WANDB_NOTEBOOK_NAME'] = 'mlops_hyperparameter_tuning.ipynb'


In [4]:
class GLUEDataModule(L.LightningDataModule):
    task_text_field_map = {
        "cola": ["sentence"],
        "sst2": ["sentence"],
        "mrpc": ["sentence1", "sentence2"],
        "qqp": ["question1", "question2"],
        "stsb": ["sentence1", "sentence2"],
        "mnli": ["premise", "hypothesis"],
        "qnli": ["question", "sentence"],
        "rte": ["sentence1", "sentence2"],
        "wnli": ["sentence1", "sentence2"],
        "ax": ["premise", "hypothesis"],
    }

    glue_task_num_labels = {
        "cola": 2,
        "sst2": 2,
        "mrpc": 2,
        "qqp": 2,
        "stsb": 1,
        "mnli": 3,
        "qnli": 2,
        "rte": 2,
        "wnli": 2,
        "ax": 3,
    }

    loader_columns = [
        "datasets_idx",
        "input_ids",
        "token_type_ids",
        "attention_mask",
        "start_positions",
        "end_positions",
        "labels",
    ]

    def __init__(
        self,
        model_name_or_path: str,
        task_name: str = "mrpc",
        max_seq_length: int = 128,
        train_batch_size: int = 32,
        eval_batch_size: int = 32,
        **kwargs,
    ):
        super().__init__()
        self.model_name_or_path = model_name_or_path
        self.task_name = task_name
        self.max_seq_length = max_seq_length
        self.train_batch_size = train_batch_size
        self.eval_batch_size = eval_batch_size

        self.text_fields = self.task_text_field_map[task_name]
        self.num_labels = self.glue_task_num_labels[task_name]
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name_or_path, use_fast=True)

    def setup(self, stage: str):
        self.dataset = datasets.load_dataset("glue", self.task_name)

        for split in self.dataset.keys():
            self.dataset[split] = self.dataset[split].map(
                self.convert_to_features,
                batched=True,
                remove_columns=["label"],
            )
            self.columns = [c for c in self.dataset[split].column_names if c in self.loader_columns]
            self.dataset[split].set_format(type="torch", columns=self.columns)

        self.eval_splits = [x for x in self.dataset.keys() if "validation" in x]

    def prepare_data(self):
        datasets.load_dataset("glue", self.task_name)
        AutoTokenizer.from_pretrained(self.model_name_or_path, use_fast=True)

    def train_dataloader(self):
        return DataLoader(self.dataset["train"], batch_size=self.train_batch_size, shuffle=True)

    def val_dataloader(self):
        if len(self.eval_splits) == 1:
            return DataLoader(self.dataset["validation"], batch_size=self.eval_batch_size)
        elif len(self.eval_splits) > 1:
            return [DataLoader(self.dataset[x], batch_size=self.eval_batch_size) for x in self.eval_splits]

    def test_dataloader(self):
        if len(self.eval_splits) == 1:
            return DataLoader(self.dataset["test"], batch_size=self.eval_batch_size)
        elif len(self.eval_splits) > 1:
            return [DataLoader(self.dataset[x], batch_size=self.eval_batch_size) for x in self.eval_splits]

    def convert_to_features(self, example_batch, indices=None):
        # Either encode single sentence or sentence pairs
        if len(self.text_fields) > 1:
            texts_or_text_pairs = list(zip(example_batch[self.text_fields[0]], example_batch[self.text_fields[1]]))
        else:
            texts_or_text_pairs = example_batch[self.text_fields[0]]

        # Tokenize the text/text pairs
        features = self.tokenizer.batch_encode_plus(
            texts_or_text_pairs, max_length=self.max_seq_length, pad_to_max_length=True, truncation=True
        )

        # Rename label to labels to make it easier to pass to model forward
        features["labels"] = example_batch["label"]

        return features

In [5]:
class GLUETransformer(L.LightningModule):
    def __init__(
        self,
        model_name_or_path: str,
        num_labels: int,
        task_name: str,
        learning_rate: float = 2e-5,
        warmup_steps: int = 0,
        weight_decay: float = 1e-5,
        train_batch_size: int = 32,
        eval_batch_size: int = 32,
        eval_splits: Optional[list] = None,
        **kwargs,
    ):
        super().__init__()

        self.save_hyperparameters()

        self.config = AutoConfig.from_pretrained(model_name_or_path, num_labels=num_labels)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, config=self.config)
        self.metric = evaluate.load(
            "glue", self.hparams.task_name, experiment_id=datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
        )

        self.validation_step_outputs = []

    def forward(self, **inputs):
        return self.model(**inputs)

    def training_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs[0]
        self.log("loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx, dataloader_idx=0):
        outputs = self(**batch)
        val_loss, logits = outputs[:2]

        if self.hparams.num_labels > 1:
            preds = torch.argmax(logits, axis=1)
        elif self.hparams.num_labels == 1:
            preds = logits.squeeze()

        labels = batch["labels"]
        self.validation_step_outputs.append({"loss": val_loss, "preds": preds, "labels": labels})
        return val_loss

    def on_validation_epoch_end(self):
        if self.hparams.task_name == "mnli":
            for i, output in enumerate(self.validation_step_outputs):
                # matched or mismatched
                split = self.hparams.eval_splits[i].split("_")[-1]
                preds = torch.cat([x["preds"] for x in output]).detach().cpu().numpy()
                labels = torch.cat([x["labels"] for x in output]).detach().cpu().numpy()
                loss = torch.stack([x["loss"] for x in output]).mean()
                self.log(f"val_loss_{split}", loss, prog_bar=True)
                split_metrics = {
                    f"{k}_{split}": v for k, v in self.metric.compute(predictions=preds, references=labels).items()
                }
                self.log_dict(split_metrics, prog_bar=True)
            self.validation_step_outputs.clear()
            return loss

        preds = torch.cat([x["preds"] for x in self.validation_step_outputs]).detach().cpu().numpy()
        labels = torch.cat([x["labels"] for x in self.validation_step_outputs]).detach().cpu().numpy()
        loss = torch.stack([x["loss"] for x in self.validation_step_outputs]).mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log_dict(self.metric.compute(predictions=preds, references=labels), prog_bar=True)
        self.validation_step_outputs.clear()

    def configure_optimizers(self):
        """Prepare optimizer and schedule (linear warmup and decay)"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay
            }
        ]

        optimizer = None
        # Initialize optimizer based on the provided hyperparameter
        if self.hparams.optimizer == "AdamW":
            optimizer_grouped_parameters[0]["eps"] = self.hparams.epsilon
            optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate)
        elif self.hparams.optimizer == "Adam":
            optimizer_grouped_parameters[0]["eps"] = self.hparams.epsilon
            optimizer = Adam(optimizer_grouped_parameters, lr=self.hparams.learning_rate)
        elif self.hparams.optimizer == "SGD":
            optimizer_grouped_parameters[0]["momentum"] = self.hparams.momentum
            optimizer = SGD(optimizer_grouped_parameters, lr=self.hparams.learning_rate)
        else:
            raise ValueError(f"Unknown optimizer type: {self.hparams.optimizer}")
    
        # Initialize learning rate scheduler
        scheduler = None
        if self.hparams.scheduler == 'linearSchedule':
            scheduler = get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps=self.hparams.warmup_steps,
                num_training_steps=self.trainer.estimated_stepping_batches,
            )
        elif self.hparams.scheduler == 'cosineSchedule':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer,
                num_warmup_steps=self.hparams.warmup_steps,
                num_training_steps=self.trainer.estimated_stepping_batches,
            )
        elif self.hparams.scheduler == 'constantSchedule':
            scheduler = get_constant_schedule_with_warmup(
                optimizer,
                num_warmup_steps=self.hparams.warmup_steps
            )
        else:
            raise ValueError(f"Unknown scheduler type: {self.hparams.scheduler}")
    
        scheduler_dict = {"scheduler": scheduler, "interval": "step", "frequency": 1}
        return [optimizer], [scheduler_dict]


In [6]:
def run_experiment(
    learning_rate: float=2e-5,
    warmup_steps: int=0,
    weight_decay: float=0,
    batch_size: int=32,
    optimizer: str="AdamW", 
    scheduler: str="linearSchedule",
    momentum: float=0.0,
    epsilon: float=1e-8,
    max_epochs = 3
):
    L.seed_everything(42)

    dm = GLUEDataModule(
        model_name_or_path="distilbert-base-uncased",
        task_name="mrpc",
    )
    dm.setup("fit")

    model = GLUETransformer(
        model_name_or_path=dm.model_name_or_path,
        num_labels=dm.num_labels,
        eval_splits=dm.eval_splits,
        task_name=dm.task_name,
        learning_rate=learning_rate,
        warmup_steps=warmup_steps,
        weight_decay=weight_decay,
        optimizer=optimizer, 
        scheduler=scheduler,
        momentum=momentum,
        epsilon=epsilon
    )
    
    wandb_logger = WandbLogger(project="hs24-mlops-p01", name=f"w2_{optimizer}_learningRate{learning_rate}_{scheduler}_warmupSteps{warmup_steps}_weightDecay{weight_decay}_batchSize{batch_size}_momentum{momentum}_epsilon{epsilon}")

    trainer = L.Trainer(
        max_epochs=max_epochs,
        accelerator="auto",
        devices=1,
        logger=wandb_logger,
        accumulate_grad_batches=batch_size / 32
    )
    trainer.fit(model, datamodule=dm)
    wandb.finish()
    

In [7]:
# optimizer
#run_experiment(optimizer="AdamW")
#run_experiment(optimizer="AdamW", epsilon=1e-10)
#run_experiment(optimizer="AdamW", epsilon=1e-6)

#run_experiment(optimizer="Adam")

#run_experiment(optimizer="SGD")
#run_experiment(optimizer="SGD", momentum=0.9)
#run_experiment(optimizer="SGD", momentum=0.2)

# learning rate
#run_experiment(learning_rate=4e-5)
#run_experiment(learning_rate=2e-4)

# weight decay
#run_experiment(weight_decay=1e-2)
#run_experiment(weight_decay=1e-4)

# batch size
#run_experiment(batch_size=64)
#run_experiment(batch_size=16)



# warmup
#run_experiment(warmup_steps=100, scheduler="linearSchedule")
#run_experiment(warmup_steps=200, scheduler="linearSchedule")
#run_experiment(warmup_steps=100, scheduler="cosineSchedule")
#run_experiment(warmup_steps=200, scheduler="cosineSchedule")
#run_experiment(warmup_steps=100, scheduler="constantSchedule")
#run_experiment(warmup_steps=200, scheduler="constantSchedule")

In [8]:
#scheduler = ["linearSchedule", "cosineSchedule"]
#warmups = [0, 200, 400]
#learning_rates = [1e-5, 2e-5, 3e-5]

#for schedule in scheduler:
    #for warmup in warmups:
        #for learning_rate in learning_rates:
            #run_experiment(scheduler=schedule, warmup_steps=warmup, learning_rate=learning_rate)

In [9]:
def tune_model(
    learning_rate: float=2e-5,
    warmup_steps: int=0,
    weight_decay: float=0,
    batch_size: int=32,
    optimizer: str="AdamW", 
    scheduler: str="linearSchedule",
    momentum: float=0.0,
    epsilon: float=1e-8,
    max_epochs = 3
):
    L.seed_everything(42)

    dm = GLUEDataModule(
        model_name_or_path="distilbert-base-uncased",
        task_name="mrpc",
    )
    dm.setup("fit")

    model = GLUETransformer(
        model_name_or_path=dm.model_name_or_path,
        num_labels=dm.num_labels,
        eval_splits=dm.eval_splits,
        task_name=dm.task_name,
        learning_rate=learning_rate,
        warmup_steps=warmup_steps,
        weight_decay=weight_decay,
        optimizer=optimizer, 
        scheduler=scheduler,
        momentum=momentum,
        epsilon=epsilon
    )
    
    wandb_logger = WandbLogger(project="hs24-mlops-p01", name=f"w3_{optimizer}_learningRate{learning_rate}_{scheduler}_warmupSteps{warmup_steps}_weightDecay{weight_decay}_batchSize{batch_size}_momentum{momentum}_epsilon{epsilon}")
    wandb_logger.watch(model)

    trainer = L.Trainer(
        max_epochs=max_epochs,
        accelerator="auto",
        devices=1,
        logger=wandb_logger,
        accumulate_grad_batches=batch_size / 32
    )
    trainer.fit(model, datamodule=dm)    

In [10]:
def train(config=None):
    with wandb.init(config):
        config = wandb.config
        tune_model(
            learning_rate=config.learning_rate,
            warmup_steps=config.warmup_steps,
            scheduler=config.scheduler,
        )

In [ ]:
sweep_config = {
    "name": "02_sweep_random_lr_warmup_scheduler",
    "method": "random",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "parameters": {
        "learning_rate": {"min": 1e-6, "max": 9e-5},
        "warmup_steps": {"min": 0, "max": 400},
        "scheduler": {"values": ["linearSchedule", "cosineSchedule"]},
    },
}
sweep_id = wandb.sweep(sweep_config, project="hs24-mlops-p01")
wandb.agent(sweep_id, function=train, count=18)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: un1gyk87
Sweep URL: https://wandb.ai/pascal-gansner-hochschule-luzern/hs24-mlops-p01/sweeps/un1gyk87


wandb: Agent Starting Run: bus1kmlk with config:
wandb: 	learning_rate: 6.431301195742979e-05
wandb: 	scheduler: cosineSchedule
wandb: 	warmup_steps: 143
wandb: Currently logged in as: hslu-pascal-gansner (pascal-gansner-hochschule-luzern). Use `wandb login --relogin` to force relogin


Seed set to 42
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.11/site-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A16') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/opt/conda/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.

  | Name  | Type                                | Params | Mode
---------------------------------------------------------------------
0 | model | DistilBertForSequenceClassification | 67.0 M | eval
---------------------------------------------------------------------
67.0 M    Trainable params
0         Non-trainable params
67.0 M    Total params
267.820   Total estimated model params size (MB)
0         Modules in train mode
96        Modules in eval mode
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_steps' w

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]